In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define the paths
validation_dir = train_dir = "C:/Users/dhana/Downloads/archive/Sports-celebrity images/Train split"
test_dir = "C:/Users/dhana/Downloads/archive/Sports-celebrity images/test split"
# = 'path_to_your_dataset/test'

# Parameters
img_height = 256
img_width = 256
batch_size = 32

# Load the training dataset
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int',
    shuffle=True
)

# Data augmentation layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomTranslation(0.2, 0.2),
    tf.keras.layers.RandomContrast(0.2)
])

# Apply data augmentation to the training dataset
augmented_train_dataset = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y)
)

# Load the validation dataset
validation_dataset = image_dataset_from_directory(
    validation_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int'
)

# Load the test dataset
test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int'
)

# Normalize the pixel values
normalization_layer = tf.keras.layers.Rescaling(1./255)
augmented_train_dataset = augmented_train_dataset.map(lambda x, y: (normalization_layer(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model
history = model.fit(
    augmented_train_dataset,
    validation_data=validation_dataset,
    epochs=17
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc}")


Found 431 files belonging to 4 classes.
Found 431 files belonging to 4 classes.
Found 4 files belonging to 4 classes.
Epoch 1/17
14/14 [==============================] - 13s 855ms/step - loss: 3.3099 - accuracy: 0.2668 - val_loss: 1.2074 - val_accuracy: 0.4965
Epoch 2/17
14/14 [==============================] - 12s 854ms/step - loss: 1.2380 - accuracy: 0.4803 - val_loss: 1.0591 - val_accuracy: 0.5568
Epoch 3/17
14/14 [==============================] - 12s 834ms/step - loss: 1.1045 - accuracy: 0.5661 - val_loss: 1.4516 - val_accuracy: 0.4664
Epoch 4/17
14/14 [==============================] - 12s 857ms/step - loss: 1.0881 - accuracy: 0.5383 - val_loss: 0.9832 - val_accuracy: 0.5592
Epoch 5/17
14/14 [==============================] - 13s 906ms/step - loss: 1.0700 - accuracy: 0.5661 - val_loss: 1.0642 - val_accuracy: 0.5290
Epoch 6/17
14/14 [==============================] - 13s 939ms/step - loss: 1.0142 - accuracy: 0.5592 - val_loss: 1.0404 - val_accuracy: 0.5708
Epoch 7/17
14/14 [======

In [24]:
import numpy as np
from sklearn.metrics import classification_report

# Get the true labels and predicted labels from the test dataset
y_true = np.concatenate([y for x, y in test_dataset], axis=0)
y_pred = np.argmax(model.predict(test_dataset), axis=-1)

class_names=['ronaldo', 'Kobe Bryant','Maria Sharapova','Kane Williamson']

# Calculate and print the classification report
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)

1/1 [==============================] - 0s 97ms/step
                 precision    recall  f1-score   support

        ronaldo       1.00      1.00      1.00         1
    Kobe Bryant       1.00      1.00      1.00         1
Maria Sharapova       0.00      0.00      0.00         1
Kane Williamson       0.00      0.00      0.00         1

       accuracy                           0.50         4
      macro avg       0.50      0.50      0.50         4
   weighted avg       0.50      0.50      0.50         4

